### RAG Pipelines - Data Ingestion to Vector DB Pipeline

In [6]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

In [7]:
def process_all_pdfs(pdf_directory):
    all_documents=[]
    pdf_dir = Path(pdf_directory)

    pdf_files = list(pdf_dir.glob("**/*.pdf"))

    print(f"Found {len(pdf_files)} PDF Files to process")

    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()

            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'

            all_documents.extend(documents)
            print(f" Loaded {len(documents)} pages")

        except Exception as e:
            print(f" Error: {e}")

    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

all_pdf_documents = process_all_pdfs("../data")

Found 3 PDF Files to process

Processing: attention.pdf
 Loaded 15 pages

Processing: embeddings.pdf
 Loaded 27 pages

Processing: objectdetection.pdf
 Loaded 21 pages

Total documents loaded: 63


In [8]:
all_pdf_documents

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '..\\data\\pdf\\attention.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1', 'source_file': 'attention.pdf', 'file_type': 'pdf'}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of T

In [9]:
def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )   
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    return split_docs
        

In [10]:
chunks=split_documents(all_pdf_documents)
chunks

Split 63 documents into 315 chunks

Example chunk:
Content: Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
...
Metadata: {'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '..\\data\\pdf\\attention.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1', 'source_file': 'attention.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '..\\data\\pdf\\attention.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1', 'source_file': 'attention.pdf', 'file_type': 'pdf'}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of T

### Embedding and VectorStoreDB

In [11]:
import numpy as np
from sentence_transformers import SentenceTransformer
# import chromadb
# from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity
import faiss
import pickle
import os
import uuid
import pickle


In [12]:
class EmbeddingManager:
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        #huggingface/all-MiniLM-L6-v2
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        if not self.model:
            raise ValueError("Model is not found")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True) 
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings
    
embedding_manager = EmbeddingManager()
embedding_manager


Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


### VectorStore

In [13]:
import os
import uuid
import pickle
from typing import List, Any

import faiss
import numpy as np


class VectorStoreFAISS:
    def __init__(
        self,
        dim: int,
        persist_dir: str = "../data/vector_store",
        store_name: str = "faiss_store",
    ):
        self.dim = dim
        self.persist_dir = persist_dir
        self.store_name = store_name

        os.makedirs(self.persist_dir, exist_ok=True)

        self.index_path = os.path.join(self.persist_dir, self.store_name + ".index")
        self.meta_path = os.path.join(self.persist_dir, self.store_name + ".pkl")

        self.index = faiss.IndexFlatL2(self.dim)

        self.documents = []
        self.metadatas = []
        self.ids = []
        self.embeddings = []

        self._load()
        print(f"FAISS Vector Store initialized. Loaded {len(self.documents)} documents.")

    def _save(self):
        faiss.write_index(self.index, self.index_path)

        data = {
            "documents": self.documents,
            "metadatas": self.metadatas,
            "ids": self.ids,
            "embeddings": self.embeddings,
        }

        with open(self.meta_path, "wb") as f:
            pickle.dump(data, f)

    def _load(self):
        if not (os.path.exists(self.index_path) and os.path.exists(self.meta_path)):
            print("No existing FAISS store found. Starting fresh.")
            return

        if os.path.getsize(self.meta_path) == 0:
            print("Metadata file empty/corrupt. Starting fresh.")
            return

        try:
            print("Loading FAISS index & metadata...")
            self.index = faiss.read_index(self.index_path)

            with open(self.meta_path, "rb") as f:
                data = pickle.load(f)

            self.documents = data.get("documents", [])
            self.metadatas = data.get("metadatas", [])
            self.ids = data.get("ids", [])
            self.embeddings = data.get("embeddings", [])

            if self.embeddings and self.index.ntotal == 0:
                arr = np.vstack(self.embeddings).astype("float32")
                self.index.add(arr)

        except Exception as e:
            print(f"Load error ({e}). Starting empty.")

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        if len(documents) != len(embeddings):
            raise ValueError("Mismatch in documents vs embeddings")

        embeddings = np.asarray(embeddings)
        if embeddings.ndim == 1:
            embeddings = embeddings.reshape(1, -1)

        embeddings = embeddings.astype("float32")

        print(f"Adding {len(documents)} documents...")

        self.index.add(embeddings)

        for i, (doc, emb) in enumerate(zip(documents, embeddings)):
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{len(self.documents) + i}"

            self.ids.append(doc_id)
            self.documents.append(doc.page_content)
            metadata = dict(getattr(doc, "metadata", {}))
            metadata["content_length"] = len(doc.page_content)
            self.metadatas.append(metadata)
            self.embeddings.append(emb)

        self._save()
        print("Done. Total docs:", len(self.documents))

    

    def count(self):
        return len(self.documents)


In [14]:
chunks

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '..\\data\\pdf\\attention.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1', 'source_file': 'attention.pdf', 'file_type': 'pdf'}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of T

In [15]:
### Text to embeddings
texts=[doc.page_content for doc in chunks]

embeddings = embedding_manager.generate_embeddings(texts)

# Initialize vectorstore
vectorstore = VectorStoreFAISS(dim=384)

vectorstore.add_documents(chunks, embeddings)

Generating embeddings for 315 texts...


Batches: 100%|██████████| 10/10 [00:40<00:00,  4.04s/it]

Generated embeddings with shape: (315, 384)
Loading FAISS index & metadata...
FAISS Vector Store initialized. Loaded 945 documents.
Adding 315 documents...
Done. Total docs: 1260


### Retriever Pipeline from VectorStore

In [32]:
class RAGRetriever:
    """Handles multi-doc retrieval from FAISS"""

    def __init__(self, vector_store: VectorStoreFAISS, embedding_manager: EmbeddingManager):
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0):
        print(f"\n🔍 Retrieving documents for query: '{query}'")

        # Embed query
        query_embedding = self.embedding_manager.generate_embeddings([query]).astype("float32")

        # Search FAISS
        distances, indices = self.vector_store.index.search(query_embedding, top_k)

        results = []
        for rank, (dist, idx) in enumerate(zip(distances[0], indices[0])):
            if idx >= len(self.vector_store.documents):
                continue

            similarity = 1 / (1 + dist)

            if similarity < score_threshold:
                continue

            results.append({
                "rank": rank + 1,
                "content": self.vector_store.documents[idx],
                "metadata": self.vector_store.metadatas[idx],
                "id": self.vector_store.ids[idx],
                "distance": float(dist),
                "similarity_score": float(similarity),
            })

        print(f"✅ Retrieved {len(results)} docs (after filtering).")
        return results


In [33]:
rag_retriever = RAGRetriever(vectorstore, embedding_manager)

query = "what is attention all you need?"
answer = rag_retriever.retrieve(query)

answer


🔍 Retrieving documents for query: 'what is attention all you need?'
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 30.98it/s]

Generated embeddings with shape: (1, 384)
✅ Retrieved 5 docs (after filtering).


[{'rank': 1,
  'content': '3.2 Attention\nAn attention function can be described as mapping a query and a set of key-value pairs to an output,\nwhere the query, keys, values, and output are all vectors. The output is computed as a weighted sum\n3',
  'metadata': {'producer': 'pdfTeX-1.40.25',
   'creator': 'LaTeX with hyperref',
   'creationdate': '2024-04-10T21:11:43+00:00',
   'author': '',
   'keywords': '',
   'moddate': '2024-04-10T21:11:43+00:00',
   'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
   'subject': '',
   'title': '',
   'trapped': '/False',
   'source': '..\\data\\pdf\\attention.pdf',
   'total_pages': 15,
   'page': 2,
   'page_label': '3',
   'source_file': 'attention.pdf',
   'file_type': 'pdf',
   'content_length': 216},
  'id': 'doc_c9ee3a27_24',
  'distance': 0.8755648136138916,
  'similarity_score': 0.5331727266311646},
 {'rank': 2,
  'content': '3.2 Attention\nAn attention function can be described

In [34]:
rag_retriever.retrieve("Unified Multi-task Learning Framework")


🔍 Retrieving documents for query: 'Unified Multi-task Learning Framework'
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 20.22it/s]

Generated embeddings with shape: (1, 384)
✅ Retrieved 5 docs (after filtering).


[{'rank': 1,
  'content': 'erage scores on CMTEB[ 22] and MTEB[ 23] benchmarks, ranking ﬁrst overall on both\nCMTEB and MTEB leaderboards, demonstrating the eﬀectiveness o f our approach.\nThe contributions of our work are summarized as follows:\n• We propose a uniﬁed multi-task learning framework that systematic ally coordi-\nnates both data processing and training pipelines, enhancing divers ity in datasets\nand eﬃciency in model training ;\n• We develop advanced data synthesis techniques powered by LLM, in cluding Para-\nphrasing, Data augmentation, and Hard negative generation. The se methods\nsigniﬁcantly enhance the quality of training corpora, thereby impro ving model’s\nrobustness and generalization capabilities;\n• We emply a two-stage training paradigm: Stage 1 focuses exclusively on retrieval\ncapability building, establishing strong foundational retrieval perf ormance; and\nstage 2 implements balanced training with controled retrieval/non-r etrieval task',
  'metadata': {'p

### RAG Pipeline - VectorDB to LLM Output Generation

In [35]:
import sys
print(sys.executable)


e:\RAG\.venv\Scripts\python.exe


In [36]:
import os
from dotenv import load_dotenv

load_dotenv()

if os.getenv("GROQ_API_KEY"):
    print("API key loaded successfully")
else:
    print("API key NOT found")


API key loaded successfully


In [37]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage

In [38]:
class GroqLLM:
    def __init__(self, model_name: str = "gemma2-9b-it", api_key: str =None):
        """
        Initialize Groq LLM
        
        Args:
            model_name: Groq model name (qwen2-72b-instruct, llama3-70b-8192, etc.)
            api_key: Groq API key (or set GROQ_API_KEY environment variable)
        """
        self.model_name = model_name
        self.api_key = api_key or os.environ.get("GROQ_API_KEY")
        
        if not self.api_key:
            raise ValueError("Groq API key is required. Set GROQ_API_KEY environment variable or pass api_key parameter.")
        
        self.llm = ChatGroq(
            groq_api_key=self.api_key,
            model_name=self.model_name,
            temperature=0.1,
            max_tokens=1024
        )
        
        print(f"Initialized Groq LLM with model: {self.model_name}")

    def generate_response(self, query: str, context: str, max_length: int = 500) -> str:
        """
        Generate response using retrieved context
        
        Args:
            query: User question
            context: Retrieved document context
            max_length: Maximum response length
            
        Returns:
            Generated response string
        """
        
        # Create prompt template
        prompt_template = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a helpful AI assistant. Use the following context to answer the question accurately and concisely.

Context:
{context}

Question: {question}

Answer: Provide a clear and informative answer based on the context above. If the context doesn't contain enough information to answer the question, say so."""
        )
        
        # Format the prompt
        formatted_prompt = prompt_template.format(context=context, question=query)
        
        try:
            # Generate response
            messages = [HumanMessage(content=formatted_prompt)]
            response = self.llm.invoke(messages)
            return response.content
            
        except Exception as e:
            return f"Error generating response: {str(e)}"
        
    def generate_response_simple(self, query: str, context: str) -> str:
        """
        Simple response generation without complex prompting
        
        Args:
            query: User question
            context: Retrieved context
            
        Returns:
            Generated response
        """
        simple_prompt = f"""Based on this context: {context}

Question: {query}

Answer:"""
        
        try:
            messages = [HumanMessage(content=simple_prompt)]
            response = self.llm.invoke(messages)
            return response.content
        except Exception as e:
            return f"Error: {str(e)}"
    


In [39]:
# Initialize Groq LLM (you'll need to set GROQ_API_KEY environment variable)
try:
    groq_llm = GroqLLM(api_key=os.getenv("GROQ_API_KEY"))
    print("Groq LLM initialized successfully!")
except ValueError as e:
    print(f"Warning: {e}")
    print("Please set your GROQ_API_KEY environment variable to use the LLM.")
    groq_llm = None

Initialized Groq LLM with model: gemma2-9b-it
Groq LLM initialized successfully!


In [40]:
### get the context from the retriever and pass it to the LLM

rag_retriever.retrieve("Unified Multi-task Learning Framework")


🔍 Retrieving documents for query: 'Unified Multi-task Learning Framework'
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 15.34it/s]

Generated embeddings with shape: (1, 384)
✅ Retrieved 5 docs (after filtering).


[{'rank': 1,
  'content': 'erage scores on CMTEB[ 22] and MTEB[ 23] benchmarks, ranking ﬁrst overall on both\nCMTEB and MTEB leaderboards, demonstrating the eﬀectiveness o f our approach.\nThe contributions of our work are summarized as follows:\n• We propose a uniﬁed multi-task learning framework that systematic ally coordi-\nnates both data processing and training pipelines, enhancing divers ity in datasets\nand eﬃciency in model training ;\n• We develop advanced data synthesis techniques powered by LLM, in cluding Para-\nphrasing, Data augmentation, and Hard negative generation. The se methods\nsigniﬁcantly enhance the quality of training corpora, thereby impro ving model’s\nrobustness and generalization capabilities;\n• We emply a two-stage training paradigm: Stage 1 focuses exclusively on retrieval\ncapability building, establishing strong foundational retrieval perf ormance; and\nstage 2 implements balanced training with controled retrieval/non-r etrieval task',
  'metadata': {'p

### Integration VectorDB Context Pipeline With LLM Output

In [41]:
### RAG Pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

### Initialize the Groq LLM (set your GROQ_API_KEY in environment)
groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(groq_api_key=groq_api_key, model_name = "llama-3.1-8b-instant", temperature=0.1, max_tokens=1024)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query, retriever, llm):
    result = retriever.retrieve(query)

    if not result or "content" not in result:
        return {"answer": "No relevant context found.", "similarity_score": 0.0}

    context = result["content"]
    score = result["similarity_score"]

    prompt = f"""
Use the following retrieved context to answer the question accurately and concisely.

Context:
{context}

Question: {query}

Answer:
"""

    response = llm.invoke(prompt)

    return {
        "answer": response.content,
        "similarity_score": score
    }


In [42]:
rag_retriever = RAGRetriever(vectorstore, embedding_manager)


In [43]:
answer = rag_simple("What is attention mechanism?", rag_retriever, llm)
print(answer)



🔍 Retrieving documents for query: 'What is attention mechanism?'
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 12.26it/s]

Generated embeddings with shape: (1, 384)
✅ Retrieved 5 docs (after filtering).
{'answer': 'No relevant context found.', 'similarity_score': 0.0}


### Enhanced RAG Pipeline Features

In [44]:
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    results = retriever.retrieve(query, top_k=top_k, score_threshold= min_score)
    if not results:
        return {'answer':'No relevant context found.', 'sources':[], 'confidence':0.0, 'context': ''}
    
    #Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page','unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    }for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])

    # Generate answer
    prompt = f"""Use the following context to answer the question accurately and concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])

    output = {
        'answer' : response.content,
        'sources':sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

result = rag_advanced("Hard Negative Mining Techniques", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])


🔍 Retrieving documents for query: 'Hard Negative Mining Techniques'
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 26.52it/s]

Generated embeddings with shape: (1, 384)
✅ Retrieved 3 docs (after filtering).


Answer: Hard negatives serve as essential components in contrastive learning for retrieval model training. Several techniques have been proposed to mine hard negatives, including:

1. **ANCE**: Proposed an asynchronous ANN indexing mechanism that periodically updates hard negatives using checkpoint states to maintain optimally challenging samples.
2. **Conan-Embedding**: Implemented a dynamic hard negative sampling strategy by excluding and refreshing samples when their scores fall below a threshold.
3. **NV-Retriever**: Proposed positive-aware negative mining, introducing TopK-MarginPos and TopKPercPos filtering criteria to minimize false negatives.
4. **LGAI-Embedding**: Built upon NV-Retriever's strategy with adaptive margin-based mining strategies, employing ANNA IR as a teacher retrieval model to identify high-quality hard negatives while using TopKPercPos filtering to eliminate false negatives.
Sources: [{'source': 'embeddings.pdf', 'page': 4, 'score': 0.6049174666404724, 'previe

In [45]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("what is attention is all you need", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])


🔍 Retrieving documents for query: 'what is attention is all you need'
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 21.15it/s]

Generated embeddings with shape: (1, 384)
✅ Retrieved 3 docs (after filtering).
Streaming answer:
Use the following context to answer the question concisely.
Context:
3.2 Attenti

on
An attention function can be described as mapping a query and a set of key-value pairs to an output,
where the query, keys, values, and output are all vector

s. The output is computed as a weighted sum
3

3.2 Attention
An attention function can be described as mapping a query and a set of key-value pairs to an output,
where the query, keys, values, and output are all vectors. The output is computed as a weighted sum
3

3.2 Attention
An attention function can be described as mapping a query and a set of key-value pairs to an output,
where the query, keys, values, and output are all vectors. The output is computed as a weighted sum
3

Question: what is attention is all you need

Answer:

Final Answer: "Attention is All You Need" is a research paper published in 2017 by Vaswani et al. that introduced the Transformer model, a type of neural network architecture that uses self-attention mechanisms to process sequential data.

Citations:
[1] attention.pdf (page 2)
[2] attention.pdf (page 2)
[3] attention.pdf (page 2)
Summary: The "Attention is All You Need" research paper, published in 2017 by Vaswani et al., introduced the Transformer model, a n